<h1> Text Classification using TensorFlow/Keras on Cloud ML Engine </h1>

This notebook illustrates:
<ol>
<li> Creating datasets for Machine Learning using BigQuery
<li> Creating a text classification model using the Estimator API with a Keras model
<li> Training on Cloud ML Engine
<li> Deploying the model
<li> Predicting with model
<li> Rerun with pre-trained embedding
</ol>

In [1]:
# change these to try this notebook out
BUCKET = 'vijays-sandbox-ml'
PROJECT = 'vijays-sandbox'
REGION = 'us-central1'
SEED = 0

In [2]:
import tensorflow as tf
print(tf.__version__) # tf 2.0 nightly
print(tf.test.is_gpu_available())

2.0.0-beta1
True


# Pure Keras

In [3]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import re
import pickle

from tensorflow.python.keras.preprocessing import sequence
from tensorflow.python.keras.preprocessing import text
from tensorflow.python.keras import models
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Embedding
from tensorflow.python.keras.layers import Conv1D
from tensorflow.python.keras.layers import MaxPooling1D
from tensorflow.python.keras.layers import GlobalAveragePooling1D

from google.cloud import storage

CLASSES = {'github': 0, 'nytimes': 1, 'techcrunch': 2}  # label-to-int mapping
TOP_K = 20000  # Limit on the number vocabulary size used for tokenization
MAX_SEQUENCE_LENGTH = 50  # Sentences will be truncated/padded to this length

"""
Helper function to download data from Google Cloud Storage
  # Arguments:
      source: string, the GCS URL to download from (e.g. 'gs://bucket/file.csv')
      destination: string, the filename to save as on local disk. MUST be filename
      ONLY, doesn't support folders. (e.g. 'file.csv', NOT 'folder/file.csv')
  # Returns: nothing, downloads file to local disk
"""
def download_from_gcs(source, destination):
    search = re.search('gs://(.*?)/(.*)', source)
    bucket_name = search.group(1)
    blob_name = search.group(2)
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(bucket_name)
    bucket.blob(blob_name).download_to_filename(destination)


"""
Parses raw tsv containing hacker news headlines and returns (sentence, integer label) pairs
  # Arguments:
      train_data_path: string, path to tsv containing training data.
        can be a local path or a GCS url (gs://...)
      eval_data_path: string, path to tsv containing eval data.
        can be a local path or a GCS url (gs://...)
  # Returns:
      ((train_sentences, train_labels), (test_sentences, test_labels)):  sentences
        are lists of strings, labels are numpy integer arrays
"""
def load_hacker_news_data(train_data_path, eval_data_path):
    if train_data_path.startswith('gs://'):
        download_from_gcs(train_data_path, destination='train.csv')
        train_data_path = 'train.csv'
    if eval_data_path.startswith('gs://'):
        download_from_gcs(eval_data_path, destination='eval.csv')
        eval_data_path = 'eval.csv'

    # Parse CSV using pandas
    column_names = ('label', 'text')
    df_train = pd.read_csv(train_data_path, names=column_names, sep='\t')
    df_eval = pd.read_csv(eval_data_path, names=column_names, sep='\t')

    return ((list(df_train['text']), np.array(df_train['label'].map(CLASSES))),
            (list(df_eval['text']), np.array(df_eval['label'].map(CLASSES))))


"""
Create tf.estimator compatible input function
  # Arguments:
      texts: [strings], list of sentences
      labels: numpy int vector, integer labels for sentences
      tokenizer: tf.python.keras.preprocessing.text.Tokenizer
        used to convert sentences to integers
      batch_size: int, number of records to use for each train batch
      mode: tf.estimator.ModeKeys.TRAIN or tf.estimator.ModeKeys.EVAL 
  # Returns:
      tf.data.dataset, produces feature and label
        tensors one batch at a time
"""
def input_fn(texts, labels, batch_size, mode):
    # Transform text to sequence of integers
    labels = tf.one_hot(labels,len(CLASSES))
    dataset = tf.data.Dataset.from_tensor_slices((texts, labels))
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return dataset.batch(batch_size)
    else: 
        return dataset.shuffle(50000).batch(batch_size)

"""
Builds a CNN model using keras and converts to tf.estimator.Estimator
  # Arguments
      model_dir: string, file path where training files will be written
      config: tf.estimator.RunConfig, specifies properties of tf Estimator
      filters: int, output dimension of the layers.
      kernel_size: int, length of the convolution window.
      embedding_dim: int, dimension of the embedding vectors.
      dropout_rate: float, percentage of input to drop at Dropout layers.
      pool_size: int, factor by which to downscale input at MaxPooling layer.
      embedding_path: string , file location of pre-trained embedding (if used)
        defaults to None which will cause the model to train embedding from scratch
      word_index: dictionary, mapping of vocabulary to integers. used only if
        pre-trained embedding is provided

    # Returns
        A tf.estimator.Estimator 
"""
def keras_estimator(model_dir,
                    config,
                    learning_rate,
                    filters=64,
                    dropout_rate=0.2,
                    embedding_dim=200,
                    kernel_size=3,
                    pool_size=3,
                    embedding_path=None,
                    word_index=None):
    # Create model instance.
    model = models.Sequential()

    # Add embedding layer
    hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1", output_shape=[128], 
                       input_shape=[], dtype=tf.string)
    model.add(hub_layer)
    #model.add(Dropout(rate=dropout_rate))
    model.add(Dense(500,activation='relu'))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(len(CLASSES), activation='softmax'))

    # Compile model with learning parameters.
    optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
    model.compile(
        optimizer=optimizer, 
        loss='categorical_crossentropy', 
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(),
            tf.keras.metrics.Recall()
        ]
    )

    return model


"""
Defines the features to be passed to the model during inference 
  Expects already tokenized and padded representation of sentences
  # Arguments: none
  # Returns: tf.estimator.export.ServingInputReceiver
"""
def serving_input_fn():
    feature_placeholder = tf.compat.v1.placeholder(tf.string, [None])
    features = feature_placeholder  # pass as-is
    return tf.estimator.export.TensorServingInputReceiver(features, feature_placeholder)

In [4]:
hparams = {'train_data_path':'./data/txtcls/train.tsv',
           'eval_data_path':'./data/txtcls/eval.tsv',
           'batch_size':128,
           'learning_rate':.001,
           'num_epochs':3}
# Load Data
((train_texts, train_labels), (test_texts, test_labels)) = load_hacker_news_data(
    hparams['train_data_path'], hparams['eval_data_path'])

model = keras_estimator(
    model_dir='output_dir',
    config=None,
    learning_rate=hparams['learning_rate'],
    embedding_path=None,
    word_index=None
)

train_dataset = input_fn(
    train_texts,
    train_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.TRAIN
)
eval_dataset = input_fn(
    test_texts,
    test_labels,
    hparams['batch_size'],
    mode=tf.estimator.ModeKeys.EVAL
)

In [5]:
%%time
tf.random.set_seed(SEED)
model.fit(
    train_dataset,
    epochs=5,
    validation_data=eval_dataset,
    validation_steps=None
)

W0814 20:17:54.764403 140494314006272 deprecation.py:323] From /usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
564/564 [==============================] - 7s 12ms/step - loss: 0.5411 - accuracy: 0.7722 - precision: 0.8011 - recall: 0.7323 - val_loss: 0.5076 - val_accuracy: 0.7833 - val_precision: 0.8050 - val_recall: 0.7570
Epoch 2/5
564/564 [==============================] - 5s 9ms/step - loss: 0.4622 - accuracy: 0.8076 - precision: 0.8276 - recall: 0.7841 - val_loss: 0.4837 - val_accuracy: 0.7966 - val_precision: 0.8145 - val_recall: 0.7741
Epoch 3/5
564/564 [==============================] - 5s 9ms/step - loss: 0.4225 - accuracy: 0.8269 - precision: 0.8430 - recall: 0.8066 - val_loss: 0.4789 - val_accuracy: 0.7996 - val_precision: 0.8152 - val_recall: 0.7791
Epoch 4/5
564/564 [==============================] - 5s 10ms/step - loss: 0.3892 - accuracy: 0.8425 - precision: 0.8571 - recall: 0.8262 - val_loss: 0.4849 - val_accuracy: 0.7987 - val_precision: 0.8131 - val_recall: 0.7826
Epoch 5/5
564/564 [==============================] - 5s 8ms/step - loss: 0.3598 - accuracy: 0.8557 - preci

In [6]:
model.save('headline_classification_model.h5')